**Start the Knowledge graph vectorDB in neo4j Desktop**

In [ ]:
import dotenv
from dotenv import load_dotenv
import os
from groq import Groq
# from langchain.schema import SystemMessage, HumanMessage
from langchain_community.graphs import Neo4jGraph
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

**Add Neo4j credentials (These information need to be kept secret)**

In [ ]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "Enter user ID"
NEO4J_PASSWORD = "Enter password"
NEO4J_DATABASE = 'neo4j'

In [ ]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

**Print the graph database schema**

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
!pip install huggingface_hub
!pip install requests
import requests
import json


### **Testing the endpoint**

In [ ]:
from langchain_core.prompt_values import StringPromptValue
# Define the Hugging Face Inference Endpoint URL and API token
endpoint_url = "Replace with your endpoint URL"
api_token = "Enter your token"

# Function to query the Hugging Face Inference Endpoint
def query_huggingface_endpoint(query):
    """#+
    This function sends a query to a Hugging Face Inference Endpoint and returns the response.#+
#+
    Parameters:#+
    query (str or StringPromptValue): The query to be sent to the Hugging Face Inference Endpoint.#+
        If the query is an instance of StringPromptValue, it will be serialized to JSON.#+
#+
    Returns:#+
    dict: The response from the Hugging Face Inference Endpoint in JSON format.#+
        If the HTTP request fails, an exception will be raised with the error details.#+
    """#+
    if isinstance(query, StringPromptValue):
        query = query.json()  # Serialize to JSON if it's a StringPromptValue

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    payload = {"inputs": query}

    # Make the HTTP request
    response = requests.post(endpoint_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")
>>>>>>> Tabnine >>>>>>># {"conversationId":"2da7cedf-b9dd-4899-83fa-5842450ef32d","source":"instruct"}

# Test the endpoint with a sample query
q_one = "I want to recycle lithium ion batteries with at least 97.5% Lithium (Li) recovery. What processes can achieve this?"
response = query_huggingface_endpoint(q_one)

# Handle and print the response
if response:
    print("LLM Response:", response)
else:
    print("No response received.")


### **Chain**

**`Simple Agent (a)`:**

### **Method 1: Final working chain**

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

# Replace with your Hugging Face inference endpoint details
endpoint_url = "Replace with your endpoint URL"
api_token = "Enter your token"


# Define the Hugging Face LLM with headers in model_kwargs
hf_llm = HuggingFaceEndpoint(
    endpoint_url=HUGGINGFACE_API_URL,
    model_kwargs={"headers": {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}}
)

# Create the Cypher chain with the custom prompt
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=hf_llm,
    qa_llm=hf_llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True
)


In [ ]:
# Test the chain
q_one = "I want to recycle lithium-ion batteries with at least 97.5% Lithium (Li) recovery, suggest which processes can achieve this?"
response = cypher_chain.invoke({"query": q_one})

# Log and debug output
print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))


### **Testing queries**

In [ ]:
import re

# Helper function to clean and reformat the generated Cypher query
def clean_cypher_query(llm_output):
    # Use regex to find the first occurrence of the "MATCH" keyword and capture the query
    match = re.search(r"(MATCH.*)", llm_output, re.DOTALL)
    if match:
        cleaned_query = match.group(1).strip()  # Extract the query starting from "MATCH"
        return f"cypher\n{cleaned_query}"  # Add "cypher" as required
    else:
        return "Invalid Cypher query: 'MATCH' keyword not found."

q_one = "I want to recycle lithium ion batteries with at least 97.5% Lithium (Li) recovery. What processes can achieve this?"
q_two = "I want to recycle Lithium-ion batteries with 100% Lithium (Li) recovery. What processes can achieve this?"
q_three = "I want to recycle Lithium-ion batteries with 95% Nickel (Ni) recovery. Which processes can achieve this in minimum cost?"
q_four = "I want to recycle lithium ion batteries and I want processes which recovers Indium element as it's expensive material, so which processes should I use?"

response = cypher_chain.invoke({"query": q_four})
#2.#response = chain._call({"query": q_one})
#3.#response = chain.invoke({"query": q_one}, callbacks=None)

#4.#from langchain_core.runnables.config import RunnableConfig
#config = RunnableConfig()
#response = chain.invoke({"query": q_one}, config=config)

#5.#response = chain.run(q_one)
# Clean and reformat the generated Cypher query
raw_cypher_query = response.get("generated_cypher", "No Cypher query generated")
cleaned_cypher_query = clean_cypher_query(raw_cypher_query)

# Print the results
print("Generated Cypher Query (Cleaned):", cleaned_cypher_query)
print("Response:", response.get("result", "No result returned"))

# If valid, execute the query
if cleaned_cypher_query.startswith("cypher\nMATCH"):
    result = graph.query(cleaned_cypher_query.split("\n", 1)[1])  # Send only the query part to Neo4j
    print("Query Result:", result)
else:
    print("Invalid or improperly formatted query:", cleaned_cypher_query)


In [ ]:


response = cypher_chain.invoke({"query": q_two})
#print(response)
#print("\nLLM response:", response["result"])

# Log and debug output
print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

In [ ]:
q_three = "which process should I use to recycle Lithium Ion batteries in minimum cost? And what are the conditions for that process"

response = cypher_chain.invoke({"query": q_three})
#print(response)
#print("\nLLM response:", response["result"])

print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

In [ ]:
q_four = "I want to recycle lithium ion batteries in minimum energy requirement, so which process should I use? And what are the conditions for that process"

response = cypher_chain.invoke({"query": q_four})
#print(response)
#print("\nLLM response:", response["result"])

print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

In [ ]:
q_five = "I want to recycle batteries. And I want processes which recovers Indium (In) element as it's an expensive material. What processes & conditions should I use?"

response = chain.invoke({"query": q_five})
#print(response)
#print("\nLLM response:", response["result"])

print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

In [ ]:
q_six = "I want to recycle batteries. And I want processes which can recover Lithium (Li), Cobalt (Co) & Manganeses (Mn) with minimum cost. What processes should I use & it's recovery rate with process conditions?"

response = chain.invoke({"query": q_six})
#print(response)
#print("\nLLM response:", response["result"])

print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

In [ ]:
q_seven = "Can you suggest battery recycling processes which recovers Cobalt (Co) & Lithium (Li) which has minimum energy requirement along with the process conditions & recovery rates?"

response = chain.invoke({"query": q_seven})
#print(response)
#print("\nLLM response:", response["result"])

print("Generated Cypher Query:", response.get("generated_cypher", "No Cypher query generated"))
print("Response:", response.get("result", "No result returned"))

### **`Special Agent`**

In [ ]:
import re
!pip install rapidfuzz
from rapidfuzz import process

q_one = "I want to recycle lithium ion batteries with at least 95% Rare Earth Elements recovery. Which processes can achieve this?"
q_two = "I want to recycle lithium ion batteries. And I want processes which recovers Indium element as it's expensive material. What processes should I use?"
q_three = "Can you suggest battery recycling processes which have 100% Cobalt (Co) recovery rate along with the conditions?"

from langchain.chains import GraphCypherQAChain

# Function to parse the user query and extract thresholds dynamically

# List of possible materials
material_list = [
    "Nickel (Ni)", "Lithium (Li)", "Cobalt (Co)", "Copper (Cu)", "Zinc (Zn)", 
    "Iron (Fe)", "Graphite (C (Graphite))", "Indium (In)", "Manganese (Mn)", 
    "Aluminium (Al)", "Cadmium (Cd)", "Phosphorus (P)", "Rare Earth Elements (REEs)"
]

# Function to extract and normalize material using fuzzy matching
def extract_material_with_fuzzy(query):
    # Clean the query (to handle abbreviations and case sensitivity)
    query_normalized = query.lower()
    # Match the query against the material list using fuzzy matching
    matched_material, score, index = process.extractOne(query_normalized, material_list)
    # You can optionally use 'score' or 'index' if needed, or just return the matched material
    return matched_material

# Updated extract_threshold_and_material function
def extract_threshold_and_material(query):
    # Extract the numeric threshold (e.g., "99.9%")
    threshold_match = re.search(r"([\d.]+)\s*%?", query, re.IGNORECASE)
    threshold = float(threshold_match.group(1)) if threshold_match else 0.0

    # Use fuzzy matching to extract the material of interest
    material = extract_material_with_fuzzy(query)
    
    return threshold, material


# Function to validate and adjust generated Cypher queries dynamically
def validate_cypher_query(cypher_query, threshold, material):
    # Dynamically replace the material in the Cypher query
    fixed_query = f"""
    MATCH (bt:Battery_Type)-[:Processed_By]->(rpu:Recycling_Process_Used)-[rel:Recovers]->(rm:Recovered_Materials)
    MATCH (rpu)-[:Has_Conditions]->(c:Process_Conditions)
    WHERE toFloat(rel.efficiency) >= {threshold} AND rm.name = "{material}"
    RETURN DISTINCT 
    rpu.name AS Process, 
    rpu.total_cost AS Cost, 
    rpu.total_energy AS Energy, 
    rel.efficiency AS Efficiency,
    c.details AS Conditions
    """
    return fixed_query

# Extract threshold and material from the user query
threshold, material = extract_threshold_and_material(q_three)

response = cypher_chain.invoke({"query": q_three})

# Debugging: Validate the generated Cypher query
generated_query = response.get("intermediate_steps", {})
print("Generated Cypher Query (Before Validation):", generated_query)

# Validate and fix the query
fixed_query = validate_cypher_query(generated_query, threshold, material)
print("Fixed Cypher Query:", fixed_query)

# Execute the fixed query directly
if fixed_query:
    context = graph.query(fixed_query)
    print("Query Results:", context)
else:
    print("\nError: No query generated or fixed query is empty.")



In [ ]:
from langchain.chains import GraphCypherQAChain

# Function to format query results into context
def format_query_results(results):
    if results:
        formatted_context = "The following recycling processes meet the specified criteria:\n\n"
        for result in results:
            process = result.get("Process", "N/A")
            efficiency = result.get("Efficiency", "N/A")
            Conditions = result.get("Conditions", "N/A")
            cost = result.get("Cost", "N/A")
            energy = result.get("Energy", "N/A")

            formatted_context += (
                f"  - Process: {process}\n"
                f"  - Efficiency: {efficiency}%\n"
                f"  - Conditions: {Conditions}\n"
                f"  - Total Cost: ${cost}\n"
                f"  - Total Energy: {energy} J\n\n"
            )
    else:
        formatted_context = "No processes meet the specified criteria."
    
    return formatted_context

# Format query results
formatted_context = format_query_results(context)
print("Formatted Context:", formatted_context)

